Used seq2seq encoder-decoder architecture, which uses LSTM (Long Short Term Memory), encodes the input hindi sequence into a single vector. This vector is then sent into the decoder neural network, which produces the English translation sentence.

#Imports 

In [ ]:
import numpy as np
import pandas as pd
import spacy
import random
from torchtext.data.metrics import bleu_score
from pprint import pprint
from torch.utils.tensorboard import SummaryWriter
from torchsummary import summary

In [ ]:
!pip install torchtext==0.6.0 --quiet
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator, TabularDataset

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.4 MB/s eta 0:00:00


'\n# Seeding for reproducible results everytime\nSEED = 777\n\nrandom.seed(SEED)\nnp.random.seed(SEED)\ntorch.manual_seed(SEED)\ntorch.cuda.manual_seed(SEED)\ntorch.backends.cudnn.deterministic = True'

In [ ]:
!nvidia-smi

Tue Apr 11 14:19:22 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8    13W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import re
from indicnlp.tokenize import indic_tokenize

In [ ]:
!python -m spacy download en_core_web_sm --quiet


2023-04-11 14:19:51.844968: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 45.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
nlp_en = spacy.load("en_core_web_sm")

In [ ]:
! pip install indic-nlp-library

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 18.8 MB/s eta 0:00:00


In [ ]:
!pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 torchaudio==0.8.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 GB 860.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 26.9 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.0+cu118
    Uninstalling torch-2.0.0+cu118:
      Successfully uninstalled torch-2.0.0+cu118
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.15.1+cu118
    Uninstalling torchvision-0.15.1+cu118:
      Successfully uninstalled torchvision-0.15.1+cu118
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.0.1+cu118
    Uninstalling torchaudio-2.0.1+cu118:
      Successfully uninstalled torchaudio-2.0.1+cu118
ERROR: pip's dependency resolver does not curre

In [ ]:

!pip install torchtext==0.6.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!git clone "https://github.com/anoopkunchukuttan/indic_nlp_library"

Cloning into 'indic_nlp_library'...
remote: Enumerating objects: 1362, done.
remote: Counting objects: 100% (143/143), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 1362 (delta 111), reused 98 (delta 93), pack-reused 1219
Receiving objects: 100% (1362/1362), 9.56 MiB | 12.04 MiB/s, done.
Resolving deltas: 100% (721/721), done.


In [ ]:
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git

Cloning into 'indic_nlp_resources'...
remote: Enumerating objects: 139, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 139 (delta 2), reused 2 (delta 0), pack-reused 126
Receiving objects: 100% (139/139), 149.77 MiB | 31.63 MiB/s, done.
Resolving deltas: 100% (53/53), done.
Updating files: 100% (28/28), done.


In [ ]:
!pip install Morfessor

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
INDIC_NLP_LIB_HOME=r"/content/indic_nlp_library"
INDIC_NLP_RESOURCES="/content/indic_nlp_resources"

In [ ]:
from indicnlp.tokenize import sentence_tokenize
from indicnlp.tokenize import indic_tokenize  

# DRIVE

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# base directory path, CS779_NLP_COMP/1_MT
base_dir = '/content/drive/MyDrive/CS779_NLP_COMP/1_MT/'

# Access files, /input_data
folder_name = 'input_data'

# file_path
file_path = base_dir + folder_name + '/'

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

# Data Pre-processing (Using torchtext and Spacy)

In [ ]:
# CS779_NLP_COMP/1_MT/input_data/eng_Hindi_data_train.csv
df_train = pd.read_csv(file_path+"eng_Hindi_data_train.csv", header = None)
df_train.head()

,0,1
0,and deliver us by Thy mercy from the people of...,और अपनी रहमत से हमें इन काफ़िर लोगों (के नीचे)...
1,Transformed position of fourth point,चौथे बिन्दु का रूपांतरित स्थान
2,"Oh, woe to me; I wish I never took so - and - ...",हाए अफसोस काश मै फला शख्स को अपना दोस्त न बनाता
3,The PS file is to be translated into a PDF fil...,पीएस2पीडीएफ के इस्तेमाल से पीएस फ़ाइल को पीडीए...
4,Receiving LDAP search results...,LDAP खोज परिणाम पा रहा है...


In [ ]:
df_train.columns = ['English', 'Hindi']
df_train.head()

,English,Hindi
0,and deliver us by Thy mercy from the people of...,और अपनी रहमत से हमें इन काफ़िर लोगों (के नीचे)...
1,Transformed position of fourth point,चौथे बिन्दु का रूपांतरित स्थान
2,"Oh, woe to me; I wish I never took so - and - ...",हाए अफसोस काश मै फला शख्स को अपना दोस्त न बनाता
3,The PS file is to be translated into a PDF fil...,पीएस2पीडीएफ के इस्तेमाल से पीएस फ़ाइल को पीडीए...
4,Receiving LDAP search results...,LDAP खोज परिणाम पा रहा है...


## Tokenization: 


In [ ]:
# Tokenize text using IndicNLP for Hindi and Spacy for English

def preprocess_text_en(text):
    # Lowercase
    text = text.lower()
    # Clean
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    doc = nlp_en(text)
    tokens = [token.text for token in doc]
    return tokens

# using IndicNLP
def preprocess_text_hi(text):
    # Lowercase
    text = text.lower()
    tokens = []
    # Clean
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    sentences= sentence_tokenize.sentence_split(text, lang='hi')
    for t_s in sentences:
      for t in indic_tokenize.trivial_tokenize(t_s):
        tokens.append(t)
    return tokens

In [ ]:
# Test Run
sample = "This is Machine Translation model"
print(preprocess_text_en(sample))

## Train/ Valid/ Test Split: 

In [ ]:
len(df_train)

140000

In [ ]:
train = df_train[:110000]
test = df_train[120000:130000]
valid = df_train[130000:]

train.to_csv('train.csv', index = False)
test.to_csv('test.csv', index = False)
valid.to_csv('valid.csv', index = False)

## Generate vocabulary
https://torchtext.readthedocs.io/en/latest/data.html

In [ ]:
# Fields for hindi and english languages
hindi = Field(tokenize=preprocess_text_hi, lower=True, init_token="<sos>", eos_token="<eos>")
english = Field(tokenize=preprocess_text_en, lower=True, init_token="<sos>", eos_token="<eos>")

In [ ]:
# Load train, validation, test data as TabularDataset
train_data = TabularDataset(path='train.csv', format='csv', fields=[('trg', english), ('src', hindi)])
valid_data = TabularDataset(path='valid.csv', format='csv', fields=[('trg', english), ('src', hindi)])
test_data = TabularDataset(path='test.csv', format='csv', fields=[('trg', english), ('src', hindi)])

In [ ]:
# Build vocabulary
hindi.build_vocab(train_data, min_freq=2)
english.build_vocab(train_data, min_freq=2)

In [ ]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Set batch size
BATCH_SIZE = 32

In [ ]:
# iterators for train, validation and test data
train_iterator, valid_iterator, test_iterator = BucketIterator.splits((train_data, valid_data, test_data), 
                                                                      batch_size = BATCH_SIZE, sort_within_batch=True, 
                                                                      sort_key=lambda x: len(x.src), device = device)

# References

https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html
<br>
https://drive.google.com/file/d/1D8dZAC3QAtAuPKfSb0qmBm1NxPcv4vUq/view
<br>
https://torchtext.readthedocs.io/en/latest/data.html